# ABS Inflation multi-measure

## Python set-up

In [1]:
# system imports
from pathlib import Path
from typing import cast

# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# local imports
from abs_data_capture import (
    AbsLandingPage,
    AbsMultiSeries,
    AbsSelectInput,
    AbsSelectionDict,
    df_from_ams,
    get_multi_series,
    rename_cols_with_stype,
)
from plotting import clear_chart_dir, finalise_plot, line_plot, set_chart_dir
from utility import qtly_to_monthly

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

### Data capture

In [2]:
wanted: AbsSelectionDict = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="2",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year",
        search2="All groups CPI ;  Australia ;",
        abbr="Q-CPI",
        calc_growth=False,
    ),
    "Qrtly CPI Trimmed Mean": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Quarter of Previous Year",
        search2="Trimmed Mean ;  Australia ;",
        abbr="Q-CPI-TM",
        calc_growth=False,
    ),
    "Monthly CPI Indicator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups CPI, seasonally adjusted ;  Australia",
        abbr="M-CPI",
        calc_growth=False,
    ),
    "Monthly CPI Trimmed Mean": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="Annual Trimmed Mean ;  Australia ;",
        abbr="M-CPI-TM",
        calc_growth=False,
    ),
    "Producer Price Index": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="producer-price-indexes-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage change from corresponding quarter of previous year",
        search2="Final ;  Total ",
        abbr="PPI",
        calc_growth=False,
    ),
    "Wage Price Index": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="wage-price-index-australia",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change From Corresponding Quarter of Previous Year",
        search2="Australia ;  Total hourly rates of pay excluding bonuses ;  "
        + "Private and Public ;  All industries ;",
        abbr="WPI",
        calc_growth=False,
    ),
    "Households final consumption Price Deflator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="national-accounts",
            topic="australian-national-accounts-national-income-expenditure-and-product",
        ),
        table="5",
        orig_sa="SA",
        search1=" ",
        search2="Households ;  Final consumption expenditure ;",
        abbr="HFCE",
        calc_growth=True,
    ),
    #    "Non-farm hourly employee compensation": AbsSelectInput(
    #        landing_page=AbsLandingPage(
    #            theme="economy",
    #            parent_topic="national-accounts",
    #            topic="australian-national-accounts-national-income-expenditure-and-product",
    #        ),
    #        table="24",
    #        orig_sa="SA",
    #        search1="Current prices ;",
    #        search2="Non-farm compensation of employees per hour:",
    #        abbr="NFHEC",
    #        calc_growth=True,
    #    ),
}

In [3]:
dataset = get_multi_series(wanted)

Retrieving data from cache: ABS_CACHE/77c08ddcb9c88a3653825f17c387e1d4--All-Time-Series-Spreadsheets.zip
Extracting DataFrames from the zip-file ...
Retrieving data from cache: ABS_CACHE/d6210953983b2a14a2f12fd9121c9a6a--All-Time-Series-Spreadsheets.zip
Extracting DataFrames from the zip-file ...
Retrieving data from cache: ABS_CACHE/8e8ca7e7459b12cae3beebd5b16f2303--Time-series-all.zip
Extracting DataFrames from the zip-file ...
Retrieving data from cache: ABS_CACHE/b7c0d5fca8c7c1c23c7c00e578c0227e--Time-series-spreadsheets-all.zip
Extracting DataFrames from the zip-file ...
Retrieving data from cache: ABS_CACHE/0e150f60e1fa66ccc2d8ce4ceb92c74f--All_time_series_workbooks.zip
Extracting DataFrames from the zip-file ...


## Plot the data

In [4]:
def plot_settings():
    """Plot settings."""

    chart_dir = "./CHARTS/Inflation"
    Path(chart_dir).mkdir(parents=True, exist_ok=True)
    clear_chart_dir(chart_dir)
    set_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


plot_settings()

In [5]:
TARGET = {
    "ymin": 2,
    "ymax": 3,
    "color": "#dddddd",
    "label": "2-3% inflation target",
    "zorder": -1,
}
LFOOTER = "Australia. Orig = Original series. SA = Seasonally adjusted series. "

In [6]:
def plot_multi_inflation(abs_data: AbsMultiSeries):
    """Produce the mulit-mesure inflation charts."""

    frame = df_from_ams(abs_data)
    source = "ABS " + ", ".join(
        [f"{x.cat_id.split(r'.', 1)[0]}-{x.table}" for x in abs_data.values()]
    )
    latest = ", ".join(
        [
            f"{abs_data[name].abbr} {frame[name].dropna().round(1).iloc[-1]}"
            for name in frame.columns
        ]
    )
    frame = rename_cols_with_stype(frame, abs_data)

    starts = ("1959-01-01", "2017-11-01")
    styles = (None, ["solid", "dotted", "dashed"] * 3)
    markers = (
        None,
        ["o", "v", "^", "<", ">", "8", "s", "p", "*", "h", "H", "D", "d", "P", "X"],
    )

    for start, style, marker in zip(starts, styles, markers):
        line_plot(
            frame,
            starts=start,
            style=style,
            marker=marker,
            markersize=6,
            dropna=True,
            title="Inflation measures",
            ylabel="Per Cent Annual Growth",
            legend={"fontsize": "xx-small"},
            axhspan=TARGET,
            y0=True,
            rfooter=source,
            lfooter=LFOOTER,
            rheader=latest if start != starts[0] else None,
            tags=start if start is not None else "",
            show=SHOW,
        )

In [7]:
plot_multi_inflation(dataset)

### Weighted CPI trajectory

In [8]:
def trajectory(abs_data: AbsMultiSeries) -> None:
    """Produce a CPI trajectory chart."""

    frame = df_from_ams(abs_data)
    frame = rename_cols_with_stype(frame, abs_data)
    thresh = 2.5
    start_year = 2019
    series_name = "Qrtly CPI Trimmed Mean (SA)"
    cpiwm_mon = frame.loc[
        cast(pd.PeriodIndex, frame.index).year >= start_year, series_name
    ].dropna()
    cpiwm_q = cpiwm_mon[cast(pd.PeriodIndex, cpiwm_mon.index).month.isin([3, 6, 9, 12])]
    cpiwm_q.index = (
        cast(pd.PeriodIndex, cpiwm_q.index).to_timestamp(how="end").to_period(freq="Q")
    )
    start = cpiwm_q[cpiwm_q > 2.5].index[0]

    peak = cast(pd.Period, cpiwm_q[cpiwm_q == cpiwm_q.max()].index[0])
    peak_minus, peak_plus = (peak - 1, peak + 1)
    tail = cast(pd.Period, cpiwm_q.index[-1])

    up_rate: float = (cpiwm_q[peak_minus] - cpiwm_q[start]) / (peak_minus - start).n
    down_rate: float = (cpiwm_q[tail] - cpiwm_q[peak_plus]) / (tail - peak_plus).n
    print(f"{up_rate=}, {down_rate=}")

    count, maximum = -1, 10
    s = pd.Series()
    while True:
        count = count + 1
        pos = peak_plus + count
        if pos <= tail:
            continue
        s[pos] = cpiwm_q[peak_plus] + (down_rate * count)
        if s[pos] <= thresh or count == maximum:
            break
    s = qtly_to_monthly(s, interpolate=False).dropna()
    ax = cpiwm_mon.plot(label=series_name)
    s.plot(ax=ax, label="Current trajectory")

    for m in [
        "Monthly CPI Trimmed Mean (Orig)",
        # "Monthly CPI Indicator (SA)",
    ]:
        cpi_mon = frame.loc[cast(pd.PeriodIndex, frame.index).year >= start_year, m]
        cpi_mon.plot(ax=ax, lw=2, label=m)

    for x in s.index:
        ax.text(x, s[x], round(s[x], 1))
    ax.text(cpiwm_mon.index[-1], cpiwm_mon.iloc[-1], round(cpiwm_mon.iloc[-1], 1))

    finalise_plot(
        axes=ax,
        title="Current underlying inflation trajectory",
        ylabel="Per cent",
        legend={
            "fontsize": "xx-small",
            "loc": "upper left",
        },
        axhspan=TARGET,
        lfooter=LFOOTER,
        rfooter="Source: ABS 6401, 6484",
        show=SHOW,
    )

In [9]:
trajectory(dataset)

up_rate=1.1666666666666665, down_rate=-0.7666666666666666


### Plot Monthly v Quarterly pairs

In [10]:
def plot_mq(ams_data: AbsMultiSeries) -> None:
    """Plot some monthly v quarterly series."""

    frame = df_from_ams(ams_data)
    frame = rename_cols_with_stype(frame, ams_data)

    data_pairs = {
        # "pair name": ["a", "b"],
        "CPI": ("Monthly CPI Indicator (SA)", "Qrtly All Groups CPI (Orig)"),
        "Trimmed Mean": (
            "Monthly CPI Trimmed Mean (Orig)",
            "Qrtly CPI Trimmed Mean (SA)",
        ),
    }

    start_year = 2019
    for name, (m, q) in data_pairs.items():
        data_m = frame.loc[cast(pd.PeriodIndex, frame.index).year >= start_year, m]
        data_q = frame.loc[cast(pd.PeriodIndex, frame.index).year >= start_year, q]
        data_q = data_q.interpolate(limit_area="inside", limit=2)
        ax = data_q.plot(label=q)
        data_m.plot(ax=ax, lw=1.5, label=m)

        finalise_plot(
            axes=ax,
            title=f"{name}: Monthly Vs Quarterly",
            ylabel="Per cent",
            legend={
                "fontsize": "xx-small",
                "loc": "upper left",
            },
            axhspan=TARGET,
            lfooter=LFOOTER,
            rfooter="Source: ABS 6401, 6484",
            y0=True,
            show=SHOW,
        )

In [11]:
plot_mq(dataset)

## Finished

In [12]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Feb 21 2024 12:37:18

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

pandas    : 2.2.0
matplotlib: 3.8.3
numpy     : 1.26.4

Watermark: 2.4.3



In [13]:
print("Finished")

Finished
